In [57]:
from selenium import webdriver # Основной модуль веб-драйвера
from selenium.webdriver.chrome.options import Options # Модуль для браузера используем Ghrome
from selenium.webdriver.support.ui import WebDriverWait # Модуль для ожидания условия
from selenium.webdriver.support import expected_conditions as EC # Модуль набора присетов для WebDriverWait
from selenium.webdriver.common.keys import Keys # Модуль симуляции клавиатуры
from selenium.webdriver.common.by import By # Определение местоположения элементов
from selenium.common.exceptions import TimeoutException, NoSuchElementException # Отлов ошибок
import time # Модуль для работы с временем
import re # Модуль для работы с регулярными выражениями
import pandas as pd # Модуль для работы с базами данных
from bs4 import BeautifulSoup # Модуль анализа HTML документов
import requests # Модуль отправки HTML запросов

In [58]:
# Модуль работы браузера
options = Options()
options.add_argument('start-maximized')  # Запуск браузера в полном окне
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36')
driver = webdriver.Chrome(options=options)  # Драйвер который запускает экземпляр браузера
driver.get('https://www.wildberries.ru/') # открытие вебсайта
time.sleep(4) # Таймер задержки

In [59]:
# Поиск строки поиска и ввод запроса в строку
wait = WebDriverWait(driver, 10) # Ожидание прогрузки страницы
try:
    input = wait.until(EC.presence_of_element_located((By.ID, "searchInput"))) # Ищем строку поиска
except (TimeoutException, NoSuchElementException, Exception) as e: 
        print("Error dont find element: {e} ")
# Вводим фразу поиска и нажимаем Enter
input.send_keys('процессор amd 9') # Имитируем ввод запроса в строку поиска
input.send_keys(Keys.ENTER) # Имитируем нажание кнопки ввода

In [60]:
# Модуль создания списка, прокручивания сайта и подсчёта карточек товара, парсинга и перехода на следующию страницу
product_list = [] # Список процессоров
# Прокручиваем сайт до конца
try:
    while True:
        count = None # Для подсчёта количества карточек товара
        while True:
            time.sleep(4) # Таймер задержки
            cards = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@id]'))) # Ищем карточку товара
            
            if len(cards) == count: # Выходим из цикла, если при прокрутке страницы, количество товаров не меняется
                break

            count = len(cards) # Посчитываем количество карточек товара на странице
            
            driver.execute_script('window.scrollBy(0, 1800)') # Прокручиваем страницу выполняя JAVA Script
            time.sleep(2) # Таймер задержки
        # Проходимся по карточкам, извлекаем ссылку на товар и добавляем в product_list    
        for card in cards:
            url = card.find_element(By.XPATH, './div/a').get_attribute('href')
            product_list.append(url)

        try:
            next = driver.find_element(By.XPATH,  "//a[@class='pagination-next pagination__next j-next-page']") # Ищем кнопку перехода на следующию страницу
            next.click()
        except (TimeoutException, NoSuchElementException, Exception): 
            break
finally:
    driver.quit() 

In [61]:
print(f'Всего получено: {len(product_list)} ссылок на процессоры AMD R9')

Всего получено: 71 ссылок на процессоры AMD R9


In [62]:
# Модуль парсинга данных с страницы товара
driver2 = webdriver.Chrome(options=options)  # Ещё один экземпляр браузера FireFox
wait2 = WebDriverWait(driver2, 10) # Таймер ожидания действий driver2
data_list = [] # Лист данных о процессорах
try:
    # Парсинг данных
    for url_item in product_list:
        data_parsing = {} # Словарь для парсинга

        driver2.get(url_item) # Переход по страницам

        # Парсим название процессора
        try:
            data_parsing['name'] = wait2.until(EC.presence_of_element_located((By.XPATH, "//h1"))).text
        except (TimeoutException, Exception):
            data_parsing['name'] = None

        # Блок парсинга цены, с скидкой, без, старой ценой, новой ценой и тд
        # Парсим цену процессора, WB кошелёк распродажа
        try:
            price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__wallet-price")]')))
            data_parsing['price_wb_wallet_sales'] = float(re.sub(r'[^\d.]+', '', price[1].text))
        except (TimeoutException, Exception):
            data_parsing['price_wb_wallet_sales'] = None

        # Парсим цену процессора, WB кошелёк старая цена
        try:
            price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__final-price wallet")]')))
            data_parsing['price_wb_wallet_old'] = float(re.sub(r'[^\d.]+', '', price[1].text))
        except (TimeoutException, Exception):
            data_parsing['price_wb_wallet_old'] = None
        
        # Парсим цену процессора, цена распродажа
        try:
            price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__final-price") and not(contains(@class, "wallet"))]')))
            data_parsing['price_sales'] = float(re.sub(r'[^\d.]+', '', price[1].text))
        except (TimeoutException, Exception):
            data_parsing['price_sales'] = None
        
        # Парсим цену процессора, старая цена до распродажи
        try:
            price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__old-price")]')))
            data_parsing['price_old'] = float(re.sub(r'[^\d.]+', '', price[1].text))
        except (TimeoutException, Exception):
            data_parsing['price_old'] = None

        # Парсим бренд процессора
        try:
            data_parsing['brand'] = wait2.until(EC.presence_of_element_located((By.CLASS_NAME, "product-page__header-brand"))).text
        except (TimeoutException, Exception):
            data_parsing['brand'] = None

        # Ссылка на процессор
        try:   
            data_parsing['url'] = url_item
        except (TimeoutException, Exception):
            data_parsing['url'] = None

        # Парсинг дополнительного элемента (Названия магазина) с помощью BeautifulSoup
        url_item = product_list
        response = driver2.page_source
        soup = BeautifulSoup(response, features="html.parser")
        site_elements = soup.find_all("div", class_="seller-info__content")
        
        try:
            shop_info = soup.select_one('div.product-page__seller-wrap:nth-child(7) > section:nth-child(2) > div:nth-child(6) > div:nth-child(1) > div:nth-child(1) > a:nth-child(2) > span:nth-child(1)')
            if shop_info:
                data_parsing['shop_name'] = shop_info.text.strip()
        except Exception as e:
            data_parsing['shop_name'] = None

        # Находим кликабельный элемент "Все характеристики и описание" чтобы получить открыть таблицу с данными
        try:
            button = WebDriverWait(driver2, 15).until(EC.element_to_be_clickable((By.CLASS_NAME, 'product-page__btn-detail')))
            if button:
                button.click() # Имитируем клик на кнопку "Все характеристики и описание"
                WebDriverWait(driver2, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="popup popup-product-details shown"]')))
        except (TimeoutException, NoSuchElementException, Exception ) as e:
            print("Error dont find element: {e} ")

        # Обрабатываем табличные данные
        table_row_name = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="popup popup-product-details shown"]//th')))
        table_row_param = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="popup popup-product-details shown"]//td')))
        # Заносим данные в зависимости от названия
        for i in range(len(table_row_name)):
            if table_row_name[i].text == 'Процессор':
                try:
                    data_parsing['processor'] = table_row_param[i].text
                except (TimeoutException, Exception):
                    data_parsing['processor'] = None
            elif table_row_name[i].text == 'Линейка процессоров':
                try:
                    data_parsing['family_processor'] = table_row_param[i].text
                except (TimeoutException, Exception):
                    data_parsing['family_processor'] = None
            elif table_row_name[i].text == 'Сокет':
                try:
                    data_parsing['soked'] = table_row_param[i].text
                except (TimeoutException, Exception):
                    data_parsing['soked'] = "-"
            elif table_row_name[i].text == 'Тактовая частота процессора':
                try:
                    val = table_row_param[i].text.strip()
                    val, *_ = val.split()
                    data_parsing['cpu_clock_speed'] = float(re.sub(r'[^\d.]+', '', val))
                except (TimeoutException, Exception):
                    data_parsing['cpu_clock_speed'] = None
            elif table_row_name[i].text == 'Максимальная частота в турбо режиме':
                try:
                    val = table_row_param[i].text.strip()
                    val, *_ = val.split()
                    data_parsing['bost_cpu_clock_speed'] = float(re.sub(r'[^\d.]+', '', val))
                except (TimeoutException, Exception):
                    data_parsing['bost_cpu_clock_speed'] = None
            elif table_row_name[i].text == 'Количество ядер процессора':
                try:
                    val = table_row_param[i].text.strip()
                    val, *_ = val.split()
                    data_parsing['processor_cores'] = float(re.sub(r'[^\d.]+', '', val))
                except (TimeoutException, Exception):
                    data_parsing['processor_cores'] = None
            elif table_row_name[i].text == 'Максимальное число потоков':
                try:
                    val = table_row_param[i].text.strip()
                    val, *_ = val.split()
                    data_parsing['max_count_threads'] = float(re.sub(r'[^\d.]+', '', val))
                except (TimeoutException, Exception):
                    data_parsing['max_count_threads'] = None
            elif table_row_name[i].text == 'Техпроцесс':
                try:
                    val = table_row_param[i].text.strip()
                    val, *_ = val.split()
                    data_parsing['technical_process'] = float(re.sub(r'[^\d.]+', '', val))
                except (TimeoutException, Exception):
                    data_parsing['technical_process'] = None
            elif table_row_name[i].text == 'Встроенная графическая система':
                try:
                    data_parsing['processor_graphic'] = table_row_param[i].text
                except (TimeoutException, Exception):
                    data_parsing['processor_graphic'] = None
            elif table_row_name[i].text == 'Объем кэша L3':
                try: 
                    val = table_row_param[i].text.strip()
                    val, *_ = val.split()
                    data_parsing['cashe_L3'] = float(re.sub(r'[^\d.]+', '', val))
                except (TimeoutException, Exception):
                    data_parsing['cashe_L3'] = None
            elif table_row_name[i].text == 'Страна производства':
                try: 
                    data_parsing['country'] = table_row_param[i].text
                except (TimeoutException, Exception):
                    data_parsing['country'] = None
            elif table_row_name[i].text == 'Гарантийный срок':
                try: 
                    data_parsing['guarantee_period'] = table_row_param[i].text
                except (TimeoutException, Exception):
                    data_parsing['guarantee_period'] = None

        data_list.append(data_parsing) # Добавляем спарсеные значения в базу
finally:
    driver2.quit() 

In [63]:
print(f'Обработано {len(data_list)} страниц')

Обработано 71 страниц


In [96]:
df = pd.DataFrame(data_list)
df.head()

,name,price_wb_wallet_sales,price_wb_wallet_old,price_sales,price_old,brand,url,shop_name,guarantee_period,processor,family_processor,soked,cpu_clock_speed,processor_cores,max_count_threads,bost_cpu_clock_speed,technical_process,cashe_L3,processor_graphic,country
0,Процессор RYZEN 9 7950X3D OEM (100-000000908),NaN,NaN,57346.0,72591.0,AMD,https://www.wildberries.ru/catalog/182999045/d...,Modern Device,1 год,7950X3D,Ryzen 9,AM5,4200.0,16.0,32.0,5700.0,5.0,128.0,AMD Radeon Graphics,Китай
1,"Ryzen 9 7900X AM5, 12 x 4700 МГц, BOX",34383.0,36193.0,NaN,61912.0,AMD,https://www.wildberries.ru/catalog/209967895/d...,СМАРТ52,P1Y,AMD Ryzen 9 7900X BOX (без кулера),AMD Ryzen 9,AM5,NaN,12.0,24.0,NaN,5.0,NaN,есть,Китай
2,Процессор RYZEN 9 5900X (100-100000061WOF ),28235.0,29722.0,NaN,37624.0,AMD,https://www.wildberries.ru/catalog/200409880/d...,Modern Device,1 год,RYZEN 9 5900X,Ryzen 9,AM4,3700.0,12.0,24.0,4800.0,7.0,64.0,нет,Китай
3,Процессор Ryzen9 7900X3D (без кулера),38874.0,40920.0,NaN,44000.0,AMD,https://www.wildberries.ru/catalog/235637677/d...,CPU HOME,1 год,Ryzen 9 7900X 3D,AMD Ryzen 9,AM5,NaN,12.0,24.0,NaN,5.0,128.0,NaN,Китай
4,Процессор Ryzen 9 7900X BOX без кулера,36023.0,37919.0,NaN,59999.0,AMD,https://www.wildberries.ru/catalog/179363514/d...,WB Retail,1 год,AMD Ryzen 9 7900X,AMD Ryzen 9,AM5,4700.0,12.0,24.0,5600.0,5.0,64.0,NaN,NaN


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   71 non-null     object 
 1   price_wb_wallet_sales  55 non-null     float64
 2   price_wb_wallet_old    55 non-null     float64
 3   price_sales            16 non-null     float64
 4   price_old              46 non-null     float64
 5   brand                  71 non-null     object 
 6   url                    71 non-null     object 
 7   shop_name              71 non-null     object 
 8   guarantee_period       62 non-null     object 
 9   processor              56 non-null     object 
 10  family_processor       57 non-null     object 
 11  soked                  61 non-null     object 
 12  cpu_clock_speed        33 non-null     float64
 13  processor_cores        59 non-null     float64
 14  max_count_threads      56 non-null     float64
 15  bost_cpu

In [98]:
data = df['processor'].value_counts()
names = data.index
values = data.values
data

processor
AMD Ryzen 9 7900X                           6
AMD Ryzen 9 7950X                           4
AMD Ryzen 9 7950X3D                         3
AMD RYZEN 9 7950X                           3
AMD Ryzen 9 5900X                           3
7950X3D                                     2
AMD RYZEN 9 7950X3D                         2
AMD Ryzen 7000 Series                       2
AMD                                         2
AMD Ryzen 9 5950X                           2
7900X                                       2
RYZEN 9 7900X3D                             2
Ryzen 9 5950X                               1
AMD RYZEN 9 7900X                           1
AMD RYZEN 7900X                             1
AMD Ryzen Threadripper 1920X                1
AMD Ryzen 9 7900X3D AM5 Новый с коробкой    1
AMD RYZEN 9 7900X3D New With Box            1
AMD Ryzen 9 7950X Новый с коробкой          1
AMD Athlon 64 2800+ ADA2800AEP4AR           1
7950X                                       1
AMD RYZEN 9 7950X Box   

In [101]:
# Небольшая обработка DataFrame перед сохранением, замена None на "-"
df = df.fillna('-').astype(object)

In [102]:
# Сохраняем DataFrame в CSV файл
filename = 'amd_processor.csv'
df = pd.DataFrame(df)
df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f'Данные успешно сохранены в файл: {filename}')

Данные успешно сохранены в файл: amd_processor.csv


1. URL сайта. Укажите URL сайта, который вы выбрали для анализа.
   - https://www.wildberries.ru/
2. Описание. Предоставьте краткое описание информации, которую вы хотели извлечь из сайта. 
   - Информация о товарах - процессорах, цена, характеристики.
3. Подход. Объясните подход, который вы использовали для навигации по сайту, определения соответствующих элементов и извлечения нужных данных. 
   - Использовался Selenium а так же BeautifulSoup для извлечения дополнительного элемента согласно заданию, затем с помощью pandas занесено в DataFrame и сохранено в csv файл.
4. Трудности. Опишите все проблемы и препятствия, с которыми вы столкнулись в ходе реализации проекта, и как вы их преодолели.
   - Браузер FireFox под который первоначально писался код не коректно работает с JS состовляющей сайта, по итогу он был заменён на Google Chrome, позже планирую доделать и решить проблему с парсингом элементов с помощью регулярных выражений для браузера FireFox, а также добавить дополнительные обработки для DataFrame.
5. Результаты. Включите образец извлеченных данных в выбранном вами структурированном формате (например, CSV или JSON).
   - amd_processor.csv
6. Примечание: Обязательно соблюдайте условия обслуживания сайта и избегайте чрезмерного скрейпинга, который может нарушить нормальную работу сайта.
   - Соблюдены но очень тяжело их соблюдать когда 95% сайтов запрещают с них что-то парсить.